# ETL Pipeline


In [19]:
import pandas as pd

messages = pd.read_csv('messages.csv', index_col='id')
messages.head(2)

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct


In [20]:
categories = pd.read_csv('categories.csv', index_col='id')
categories.head(2)

,categories
id,
2,related-1;request-0;offer-0;aid_related-0;medi...
7,related-1;request-0;offer-0;aid_related-1;medi...


#### Merge datasets

In [21]:
df = messages.merge(categories, on='id')
df.head(2)

,message,original,genre,categories
id,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...


#### Split "categories" into separate category columns

In [22]:
cols = [string[0:-2] for string in df.head(1).categories.values[0].split(';')]
categories = df.categories.str.split(';', expand=True)
categories.columns = cols

In [23]:
categories.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


#### Convert category values to 0s and 1s

In [24]:
for column in categories:
    categories[column] = categories[column].str.slice(-1)
    categories[column] = pd.to_numeric(categories[column])

In [26]:
categories.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


#### Replace "categories" in df with categories dataframe


In [27]:
df.shape, categories.shape

((26386, 4), (26386, 36))

In [28]:
df = pd.concat([df.drop('categories', axis=1), categories], axis=1)

In [30]:
df.shape

(26386, 39)

In [29]:
df.head(2)

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0


#### Remove duplicates.

In [31]:
df.drop(df[df.duplicated()].index, axis=0, inplace=True)

In [32]:
df.shape

(26112, 39)

#### Save the clean dataset into an sqlite database.

In [15]:
!pip install SQLAlchemy

In [16]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///DisasterResponse.db', encoding='UTF-8')
df.to_sql('messages_and_categories', engine, index=False, if_exists='replace')